In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np

import time
from autoemulate.datasets import fetch_data
from autoemulate.compare import AutoEmulate
import matplotlib.pyplot as plt

# 1. simple gp-like data

In [ ]:
def make_gp_data(n_points, x_range=(0, 10), num_basis=20, length_scale=0.5, noise_std=0.1):
    def simple_gp_like_function(x):
        centers = np.random.uniform(x_range[0], x_range[1], num_basis)
        weights = np.random.randn(num_basis)
        dist = np.abs(x[:, np.newaxis] - centers)
        rbf = np.exp(-(dist**2) / (2 * length_scale**2))
        return rbf.dot(weights)
    X = np.random.uniform(x_range[0], x_range[1], n_points)
    y = simple_gp_like_function(X) + np.random.normal(0, noise_std, n_points)
    
    return X.reshape(-1, 1), y

# Generate data
n_points = 400
X, y = make_gp_data(n_points)

In [ ]:
plt.figure(figsize=(5, 3))  # Set a smaller figure size
plt.plot(X, y, ".")

In [ ]:
em = AutoEmulate()
em.setup(X, y, print_setup=False)
em.compare()
em.print_results()

In [ ]:
em.plot_results(model="gpt")

# 2. difficult data for gp's

In [ ]:
def generate_challenging_data(n_samples=1000, noise_level=0.01):
    """
    Generate X, y data that's challenging for a Gaussian Process to fit.
    
    Parameters:
    n_samples (int): Number of samples to generate
    noise_level (float): Standard deviation of Gaussian noise to add
    
    Returns:
    X (np.array): 2D array of shape (n_samples, 1)
    y (np.array): 2D array of shape (n_samples, 1)
    """
    X = np.linspace(0, 10, n_samples).reshape(-1, 1)
    y = np.zeros((n_samples, 1))
    
    # Create a function with abrupt changes and discontinuities
    y = np.where(X < 2, np.sin(2 * np.pi * X), y)
    y = np.where((X >= 2) & (X < 4), 2 * np.exp(-((X - 3)**2)), y)
    y = np.where((X >= 4) & (X < 6), -1.5 * np.ones_like(X), y)
    y = np.where((X >= 6) & (X < 8), 0.5 * np.cos(3 * np.pi * (X - 7)), y)
    y = np.where(X >= 8, np.abs(X - 9), y)
    
    # Add some high-frequency oscillations
    y += 0.05 * np.sin(20 * np.pi * X)
    # Add Gaussian noise
    y += np.random.normal(0, noise_level, (n_samples, 1))
    
    return X, y

X, y = generate_challenging_data(n_samples=600, noise_level=0.0)

In [ ]:
plt.figure(figsize=(5, 3))  # Set a smaller figure size
plt.plot(X, y, ".")

In [ ]:
em = AutoEmulate()
em.setup(X, y, print_setup=False)
em.compare()
em.print_results()

In [ ]:
em.plot_results()

# 3. projectile motion with drag
from https://mogp-emulator.readthedocs.io/en/latest/

In [ ]:
from autoemulate.simulations.projectile import simulate_projectile_multioutput
from autoemulate.experimental_design import LatinHypercube
lhd = LatinHypercube([(-5., 1.), (0., 1000.)])
X = lhd.sample(150)
y = np.array([simulate_projectile_multioutput(x) for x in X])

In [ ]:
em = AutoEmulate()
em.setup(X, y, print_setup=False)
em.compare()
em.print_results()

In [ ]:
em.plot_results()

# 4. engineering: cantilever beam

In [ ]:
X, y = fetch_data('engineering1')
X.shape, y.shape

In [ ]:
em = AutoEmulate()
em.setup(X, y, print_setup=False)
em.compare()
em.print_results()

In [ ]:
em.plot_results()

# 5. cardiac: atrial cell model

In [ ]:
X, y = fetch_data('cardiac1')
X.shape, y.shape

In [ ]:
em = AutoEmulate()
em.setup(X, y, print_setup=False)
em.compare()
em.print_results()

In [ ]:
em.plot_results()

# 6. cardiac: isotonic contraction

In [ ]:
X, y = fetch_data('cardiac2')
X.shape, y.shape

In [ ]:
em = AutoEmulate()
em.setup(X, y, print_setup=False)
em.compare()
em.print_results()

In [ ]:
em.plot_results("cnp")

# 7. cardiac: circ adapt model

In [ ]:
X, y = fetch_data('cardiac3')
X.shape, y.shape
em = AutoEmulate()
em.setup(X, y, print_setup=False)
em.compare()
em.print_results()
em.plot_results()

# 8. cardiac: four chamber model

In [ ]:
X, y = fetch_data('cardiac4')
print(X.shape, y.shape)
em = AutoEmulate()
em.setup(X, y, print_setup=False, param_search=True, n_jobs=6)
em.compare()
em.print_results()

In [ ]:
X, y = fetch_data('cardiac4')
print(X.shape, y.shape)
em = AutoEmulate()
em.setup(X, y, print_setup=False)
em.compare()
em.print_results()

In [ ]:
em.plot_results()

# 8. cardiac: passive mechanics model

In [ ]:
X, y = fetch_data('cardiac5')
em = AutoEmulate()
em.setup(X, y, print_setup=False)
em.compare()
em.print_results()

In [ ]:
em.plot_results(model="cnp")

# 9. cardiac: tissue electrophysiology

In [ ]:
X, y = fetch_data('cardiac6')
em = AutoEmulate()
em.setup(X, y, print_setup=False)
em.compare()
em.print_results()

In [ ]:
em.plot_results()

# 10. climate model

In [ ]:
X, y = fetch_data('climate1')
em = AutoEmulate()
em.setup(X, y, print_setup=False)
em.compare()
em.print_results()

In [ ]:
em.plot_results()